In [8]:
import pandas as pd
import json
from pandas import json_normalize
from os import getcwd, path
from yaml import SafeLoader, load
import yaml
import jmespath

import yaml
import jmespath


In [9]:
df_from_old=pd.read_csv("/root/main-kafka-main/DataFrame/4412d9e0-ab8c-11ec-bcd8-afb5f342d01b-20250225_113711.csv")

In [10]:
df_from_old.head()

,name,id,type,label,customerId,CustomerName
0,GKC-A848FA86D8E4-002,0571c980-270f-11ee-bc9b-1d7b9ef08863,Modbus V6 Fan,AB-08 Venti 4,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume
1,GKC-A848FA87769C-021,074033d0-92c7-11ef-bb66-09a77fdcecde,Thermokon,NaN,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume
2,GKC-A848FA86D8E4-003,0c7067f0-270f-11ee-bc9b-1d7b9ef08863,Modbus V6 Fan,AB-08 Venti 3,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume
3,GKC-A848FA870B50-013,0d876b50-2711-11ee-927a-b9de331472ce,Modbus V6 Fan,ZU-10 Venti 1,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume
4,GKC-A848FA86D8E4-004,16a5ee70-270f-11ee-927a-b9de331472ce,Modbus V6 Fan,AB-08 Venti 2,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume


In [11]:
df_from_old["CustomerName"].value_counts()

CustomerName
Daume    65
Name: count, dtype: int64

In [36]:
df_from_old.isna().sum()

name             0
id               0
type             0
label           15
customerId       0
CustomerName     0
dtype: int64

#  Getting value from JAML  file

In [13]:
data="/root/main-kafka-main/Emb-yaml/epStructure2.yaml"
device_names_list=[]
with open(data, "r") as file:
    yaml_content = yaml.safe_load(file)
    projects=yaml_content["projects"]
    for project in projects:
        costomer=project["customer"]
        device_names = jmespath.search("sites[*].locations[*].systems[*].devices[*].name", project) 
        device_names_list.append(device_names)

In [14]:

customer_devices = {} 
with open(data, "r") as file:
    yaml_content = yaml.safe_load(file)
    projects = yaml_content["projects"]
    for project in projects:
        customer = project["customer"]
        if customer not in customer_devices:
            customer_devices[customer] = []
        device_names_result = jmespath.search("sites[*].locations[*].systems[*].devices[*].name", project)
        for level1 in device_names_result:
            for level2 in level1:
                for level3 in level2:
                    for name in level3:
                        if name is not None:
                            customer_devices[customer].append(name)
        

In [15]:
devices_df = pd.DataFrame(customer_devices)


In [16]:
devices_df=pd.melt(devices_df,value_vars=['HaustechnikRCP']).rename(columns={"variable":"CustomerNameNew", "value":"name"})

In [17]:
devices_df.head()

,CustomerNameNew,name
0,HaustechnikRCP,GKC-A848FA86D8E4-002
1,HaustechnikRCP,GKC-A848FA86D8E4-003
2,HaustechnikRCP,GKC-A848FA86D8E4-004
3,HaustechnikRCP,GKC-A848FA86D8E4-005
4,HaustechnikRCP,GKC-A848FA86D8E4-016


In [37]:
devices_df.isna().sum()

CustomerNameNew    0
name               0
dtype: int64

In [18]:
devices_df.shape

(39, 2)

In [19]:
df_from_old.shape

(65, 6)

In [20]:
df_from_old["name"] = df_from_old["name"].astype(str)
devices_df["name"] = devices_df["name"].astype(str)

In [21]:

both_have = df_from_old.join(devices_df.set_index("name"), on="name", how="inner")

In [22]:
both_have.head()

,name,id,type,label,customerId,CustomerName,CustomerNameNew
0,GKC-A848FA86D8E4-002,0571c980-270f-11ee-bc9b-1d7b9ef08863,Modbus V6 Fan,AB-08 Venti 4,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,HaustechnikRCP
2,GKC-A848FA86D8E4-003,0c7067f0-270f-11ee-bc9b-1d7b9ef08863,Modbus V6 Fan,AB-08 Venti 3,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,HaustechnikRCP
3,GKC-A848FA870B50-013,0d876b50-2711-11ee-927a-b9de331472ce,Modbus V6 Fan,ZU-10 Venti 1,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,HaustechnikRCP
4,GKC-A848FA86D8E4-004,16a5ee70-270f-11ee-927a-b9de331472ce,Modbus V6 Fan,AB-08 Venti 2,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,HaustechnikRCP
8,GKC-A848FA870B50-002,25327d90-2710-11ee-927a-b9de331472ce,Modbus V6 Fan,AB-10 Venti 8,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,HaustechnikRCP


In [23]:
both_have.shape

(39, 7)

# Devices that exist in the old system but not in the new system


In [24]:
full_old= df_from_old.join(devices_df.set_index("name"), on="name", how="left")

In [25]:
full_old.head()

,name,id,type,label,customerId,CustomerName,CustomerNameNew
0,GKC-A848FA86D8E4-002,0571c980-270f-11ee-bc9b-1d7b9ef08863,Modbus V6 Fan,AB-08 Venti 4,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,HaustechnikRCP
1,GKC-A848FA87769C-021,074033d0-92c7-11ef-bb66-09a77fdcecde,Thermokon,NaN,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,NaN
2,GKC-A848FA86D8E4-003,0c7067f0-270f-11ee-bc9b-1d7b9ef08863,Modbus V6 Fan,AB-08 Venti 3,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,HaustechnikRCP
3,GKC-A848FA870B50-013,0d876b50-2711-11ee-927a-b9de331472ce,Modbus V6 Fan,ZU-10 Venti 1,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,HaustechnikRCP
4,GKC-A848FA86D8E4-004,16a5ee70-270f-11ee-927a-b9de331472ce,Modbus V6 Fan,AB-08 Venti 2,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,HaustechnikRCP


In [26]:
print(f"There are {full_old['CustomerNameNew'].isna().sum()} devices that exist in the old system but not in the new system.")


There are 26 devices that exist in the old system but not in the new system.


In [27]:
full_old["CustomerNameNew"].isna().sum()

np.int64(26)

In [28]:
full_old[full_old["CustomerNameNew"].isna()]

,name,id,type,label,customerId,CustomerName,CustomerNameNew
1,GKC-A848FA87769C-021,074033d0-92c7-11ef-bb66-09a77fdcecde,Thermokon,NaN,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,NaN
5,GKC-A848FA87769C-012,1c44db00-92c2-11ef-b675-81d50ee23e1f,Modbus V6 Fan,ZU-15 Venti 3,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,NaN
6,GKC-A848FA87769C-011,1c4e9f00-92c2-11ef-b675-81d50ee23e1f,Modbus V6 Fan,ZU-15 Venti 2,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,NaN
7,GKC-A848FA87769C-010,1c566730-92c2-11ef-b675-81d50ee23e1f,Modbus V6 Fan,ZU-15 Venti 1,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,NaN
10,GKC-A848FA86C458,2906b580-4af2-11ed-9a85-43b5866161ae,IntelliGateAIR,NaN,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,NaN
12,GKC-A848FA880220,32ccc230-8beb-11ef-bb66-09a77fdcecde,IntelliGateAIR,NaN,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,NaN
13,GKC-A848FA8784F4,33724420-9afe-11ec-bcd8-afb5f342d01b,IntelliGateAIR,NaN,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,NaN
15,GKC-A848FA87769C-002,3c247f30-92b7-11ef-bb66-09a77fdcecde,Modbus V6 Fan,AB-15 Venti 1,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,NaN
24,GKC-A848FA870B50,3d3d4ac0-0c2e-11ee-a50f-b5cb373dcb6a,IntelliGateAIR,NaN,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,NaN
26,GKC-A848FA87769C-003,4010ad80-92b7-11ef-bb66-09a77fdcecde,Modbus V6 Fan,AB-15 Venti 2,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,NaN


# Devices that exist NOT in the old system but in the new system

In [29]:
full_new= df_from_old.join(devices_df.set_index("name"), on="name", how="right")
full_new.shape

(39, 7)

In [30]:
full_new.head()

,name,id,type,label,customerId,CustomerName,CustomerNameNew
0,GKC-A848FA86D8E4-002,0571c980-270f-11ee-bc9b-1d7b9ef08863,Modbus V6 Fan,AB-08 Venti 4,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,HaustechnikRCP
2,GKC-A848FA86D8E4-003,0c7067f0-270f-11ee-bc9b-1d7b9ef08863,Modbus V6 Fan,AB-08 Venti 3,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,HaustechnikRCP
4,GKC-A848FA86D8E4-004,16a5ee70-270f-11ee-927a-b9de331472ce,Modbus V6 Fan,AB-08 Venti 2,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,HaustechnikRCP
9,GKC-A848FA86D8E4-005,28d3dcb0-270f-11ee-bc9b-1d7b9ef08863,Modbus V6 Fan,AB-08 Venti 1,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,HaustechnikRCP
33,GKC-A848FA86D8E4-016,54711570-2c7f-11ee-863a-01bce6d27c63,Thermokon,NaN,4412d9e0-ab8c-11ec-bcd8-afb5f342d01b,Daume,HaustechnikRCP


In [31]:
full_new["CustomerName"].isna().sum()

np.int64(0)

In [32]:
print(f"There are {full_new['CustomerNameNew'].isna().sum()} devices that exist NOT in the old system but  in the new system.")


There are 0 devices that exist NOT in the old system but  in the new system.


In [33]:
full_new[full_new["CustomerNameNew"].isna()]

,name,id,type,label,customerId,CustomerName,CustomerNameNew


# Save the data

In [34]:
df_final=both_have['id']

In [35]:
df_final.to_csv('Datasets/temp.csv',  header =False, index=False)


OSError: Cannot save file into a non-existent directory: 'Datasets'